In [1]:
import logging
from pprint import pprint
from resimulate.euicc.card import Card
from resimulate.euicc.exceptions import IccidAlreadyExists
from resimulate.euicc.models.activation_profile import ActivationProfile
from resimulate.euicc.models.reset_option import ResetOption
from resimulate.euicc.mutation.deterministic_engine import DeterministicMutationEngine
from resimulate.euicc.recorder.recorder import OperationRecorder
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

mutation_engine = DeterministicMutationEngine()
recorder = OperationRecorder()

profile = ActivationProfile.from_activation_code(
    "LPA:1$rsp.truphone.com$QR-G-5C-1LS-1W1Z9P7"
)

print(profile)

with PcscLink(recorder=recorder, apdu_data_size=255) as link:
    card = Card(link)
    print([app.name for app in card.supported_applications.values()])
    try:
        print(card.isd_r.get_euicc_challenge())
    except Exception:
        pass
    print(card.isd_r.get_euicc_info_2())
    print(card.isd_r.get_euicc_info_1())
    print(card.isd_r.get_eid())
    try:
        notification = card.isd_r.download_profile(profile)
        iccid = notification.data.notification.iccid
        pprint(notification)
        card.isd_r.process_notifications([notification])
        card.isd_r.set_nickname(iccid, "Some Nickname")
        card.isd_r.enable_profile(iccid)
        card.isd_r.disable_profile(iccid)
    except IccidAlreadyExists:
        pprint("Profile already exists")
        pass

    profiles = card.isd_r.list_profiles()
    pprint(profiles)
    for profile in profiles:
        # card.isd_r.delete_profile(isdp_aid=profile.isdp_aid)
        notifications = card.isd_r.retrieve_notification_list()
        card.isd_r.process_notifications(notifications)

    card.isd_r.set_default_dp_address("some random address")
    card.isd_r.reset_euicc_memory(ResetOption.RESET_DEFAULT_SMDP_ADDRESS)

DEBUG:root:Initializing recorder...
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1


ActivationProfile(smdpp_address='rsp.truphone.com', matching_id='QR-G-5C-1LS-1W1Z9P7', confirmation_code=None)


DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sent TPDU: 00a4040407a0000000871002, received data: , SW: 6138
DEBUG:root:Sent TPDU: 00c0000038, received data: 62368202782183027ff08410a0000000871002ffffffff89030500018a01058b032f060cc60c90016083010183018183010a8104000784a7, SW: 9000
DEBUG:root:Selected ADF a0000000871002
DEBUG:root:Found USIM via ADF a0000000871002 with CLA byte 00
DEBUG:root:Sent TPDU: 00a4040410a0000005591010ffffffff8900000100, received data: , SW: 6a82
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000100! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Sent TPDU: 01a4040410a0000005591010ffffffff8900000100, received data: , SW: 6999
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000100! Failed with SW match failed! Expected 9000 and got 6999.
DEBUG:root:Sent TPDU: 80a4040410a0000005591010ffffffff8900000100, received data: , SW: 6a82
DEBUG:root:Error sele

['USIM', 'ISDR', 'ECASD']
a46e54d97845fad91bde03df2eae95de
{'profileVersion': b'\x02\x03\x01', 'svn': b'\x02\x02\x02', 'euiccFirmwareVer': b'\x04\x02\x00', 'extCardResource': b'\x81\x01\x00\x82\x04\x00\x07YD\x83\x0273', 'uiccCapability': (bytearray(b'\x7f>\x1f\x80'), 25), 'ts102241Version': b'\t\x02\x00', 'globalplatformVersion': b'\x02\x03\x00', 'rspCapability': (bytearray(b'\x90'), 4), 'euiccCiPKIdListForVerification': [b'\x0b\x13Yc=*\xb4iv\x1c\xa9\xd8+")5\r\xb7"\xd7', b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb'], 'uiccCiPKIdListForSigning': [b'\x0b\x13Yc=*\xb4iv\x1c\xa9\xd8+")5\r\xb7"\xd7', b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb'], 'euiccCategory': 0, 'ppVersion': b'\x01\x00\x00', 'sasAcreditationNumber': 'ED-ZI-UP-0824'}
{'svn': b'\x02\x02\x02', 'euiccCiPKIdListForVerification': [b'\x0b\x13Yc=*\xb4iv\x1c\xa9\xd8+")5\r\xb7"\xd7', b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^y[\xeb\xfb'], 'euiccCiPKIdListForSigning': [b'\x0b\x13Yc=*\xb4

DEBUG:root:Downloading profile from rsp.truphone.com
DEBUG:root:No mutation engine provided, sending original APDU
DEBUG:root:Sending APDU (cla=80 ins=e2 p1=91 p2=00 lc=03 data=bf2e00 le=00 extended=False)
DEBUG:root:Sent TPDU: 80e2910003bf2e00, received data: , SW: 6115
DEBUG:root:Sent TPDU: 80c0000015, received data: bf2e12801016a64499bb49422497ff1e22cc91c984, SW: 9000
DEBUG:root:No mutation engine provided, sending original APDU
DEBUG:root:Sending APDU (cla=80 ins=e2 p1=91 p2=00 lc=03 data=bf2000 le=00 extended=False)
DEBUG:root:Sent TPDU: 80e2910003bf2000, received data: , SW: 6164
DEBUG:root:Sent TPDU: 80c0000064, received data: bf20618203020202a92c04140b1359633d2ab469761ca9d82b2229350db722d7041481370f5125d0b1d408d4c3b232e6d25e795bebfbaa2c04140b1359633d2ab469761ca9d82b2229350db722d7041481370f5125d0b1d408d4c3b232e6d25e795bebfb, SW: 9000
DEBUG:httpcore.connection:connect_tcp.started host='rsp.truphone.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.co

ApduException: 6a88 -> Referenced data not found

In [3]:
from pprint import pprint
import logging

from resimulate.euicc.card import Card
from resimulate.euicc.recorder.recorder import OperationRecorder
from resimulate.euicc.transport.pcsc_link import PcscLink

logging.basicConfig(level=logging.DEBUG)

pprint(
    OperationRecorder.compare_files(
        "estk.resim", ["5ber.resim", "eiot.resim", "esim_me.resim"]
    )
)

DEBUG:root:Loaded compare recording: 5ber.resim with 4 mutations
DEBUG:root:Loaded compare recording: eiot.resim with 4 mutations
DEBUG:root:Loaded compare recording: esim_me.resim with 4 mutations
DEBUG:root:Comparing estk.resim with 5ber.resim
DEBUG:root:Comparing estk.resim with eiot.resim
DEBUG:root:Comparing estk.resim with esim_me.resim


[]


In [2]:
bytes.fromhex("02")

b'\x02'

In [2]:
with PcscLink(recorder=recorder) as link:
    card = Card(link)
    recorder.replay("test_get_euicc_info.resim", card)

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Error selecting ADF A0000005591010FFFFFFFF8900000300! Failed with SW match failed! Expected 9000 and got 6a82.
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Replaying operation: get_euicc_challenge with mutation None
DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00


AttributeError: 'Operation' object has no attribute 'original_args'

In [2]:
import logging

from resimulate.scenarios import SCENARIOS
from resimulate.scenarios.models.scenario_runner import ScenarioRunner

logging.basicConfig(level=logging.DEBUG)

runner = ScenarioRunner(scenarios=SCENARIOS)
runner.run_scenarios()

DEBUG:root:Disconnected from device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Connecting to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00 using protocol T1
DEBUG:root:Connected to device HID Global OMNIKEY 3x21 Smart Card Reader [OMNIKEY 3x21 Smart Card Reader] 00 00
DEBUG:root:Sent TPDU: 00a4040407a0000000871002, received data: , SW: 613f
DEBUG:root:Sent TPDU: 00c000003f, received data: 623d8202782183027fd0840ca0000000871002ff49ff0589a50c80017183040016d1208701018a01058b032f0601c60f90017083010183018183010a83010b, SW: 9000
DEBUG:root:Sent TPDU: 00a4040410A0000005591010FFFFFFFF8900000100, received data: , SW: 6121
DEBUG:root:Sent TPDU: 00c0000021, received data: 6f1f8410a0000005591010ffffffff8900000100a5049f6501ffe0058203020300, SW: 9000
DEBUG:root:Sent TPDU: 00a4040410A0000005591010FFFFFFFF8900000200, received data: , SW: 611a
DEBUG:root:Sent TPDU: 00c000001a, received data: 6f188410a000000559101

In [2]:
import asn1tools

rsp_v2_6 = asn1tools.compile_files(
    [
        "/home/niklas/Documents/documents/uni/master_thesis/resimulate/asn/pkix1_explicit_88.asn",
        "/home/niklas/Documents/documents/uni/master_thesis/resimulate/asn/pkix1_implicit_88.asn",
        "/home/niklas/Documents/documents/uni/master_thesis/resimulate/asn/pe_definitions_v3_4.asn",
        "/home/niklas/Documents/documents/uni/master_thesis/resimulate/asn/rsp_definitions_v2_6.asn",
    ],
    codec="ber",
    cache_dir="asn_cache",
)

In [5]:
import base64

authenticate_server_response = "vziCBeigggXkMIHJgAgAAAAAGNjLsoMQcnNwLnRydXBob25lLmNvbYQQa/Ugq/w3o10ewVKyNwVpQ78id4EDAgMBggMCAwCDAyQRBIQNgQELggQAF1IAgwJ/44UFBn8298CGAw8BAIcDAgMAiAIDmKkWBBSBNw9RJdCx1AjUw7Iy5tJeeVvr+6oWBBSBNw9RJdCx1AjUw7Iy5tJeeVvr+wQDAQAADA1LTi1ETi1VUC0wOTI0oB+AE1FSLUctNUMtMUxTLTFXMVo5UDehCIAENSkGEaEAXzdAUzHVWCNLuyt5pDB6GQb4jFdcoj5/oAxegG7jj46J8XDYgJzxms+uLc70lkIlrfCLc9HAgpKWlXGqocsWTh3wajCCAgcwggGsoAMCAQICDgRj5NPd16wGH8wb7ZorMAoGCCqGSM49BAMCMGIxCzAJBgNVBAYTAlVLMRIwEAYDVQQHDAlDYW1icmlkZ2UxDjAMBgNVBAoMBUtpZ2VuMRQwEgYDVQQLDAtFbmdpbmVlcmluZzEZMBcGA1UEAwwQRVVNQ29uc3VtZXJER0R1YjAgFw0yNDExMTIwODMxMTdaGA8yMTI0MTExMjA4MzExN1owOzEOMAwGA1UECgwFS2lnZW4xKTAnBgNVBAUTIDg5MDQ0MDQ1MjE2NzI3NDk0ODAwMDAwMDAxMzUwMTg3MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEqt+MxSB28dwnDlymxPJurDV+ulj7VdM2cQQHGQDg8u33HNd//+POlr7FnJu3D7+P5FutkbX6da0HATAKAIevHKNrMGkwHQYDVR0OBBYEFJ8DsA/PJVMFhZJtcw1QAzYWtBOaMB8GA1UdIwQYMBaAFBiaCFOv1JT2sHmaC2fw9dDBKeP2MA4GA1UdDwEB/wQEAwIHgDAXBgNVHSABAf8EDTALMAkGB2eBEgECAQEwCgYIKoZIzj0EAwIDSQAwRgIhAOQGFqozk99v9eE13TWQWtTN4Ytwstk2mYZLBIwzujX7AiEAgR96Fg7h9DDqpzBiiLChSF8eudV6SpLzABtCi6NZL3gwggLGMIICbaADAgECAhBazo+Y3IZJQEq9rWV+IFCTMAoGCCqGSM49BAMCMEQxGDAWBgNVBAoTD0dTTSBBc3NvY2lhdGlvbjEoMCYGA1UEAxMfR1NNIEFzc29jaWF0aW9uIC0gUlNQMiBSb290IENJMTAeFw0yMjEwMDUwMDAwMDBaFw00OTEyMzEyMzU5NTlaMGIxCzAJBgNVBAYTAlVLMRIwEAYDVQQHDAlDYW1icmlkZ2UxDjAMBgNVBAoMBUtpZ2VuMRQwEgYDVQQLDAtFbmdpbmVlcmluZzEZMBcGA1UEAwwQRVVNQ29uc3VtZXJER0R1YjBZMBMGByqGSM49AgEGCCqGSM49AwEHA0IABMb5fuovdorRe6himrAE7PeQ/0NX8ggb4VwjGfcuyWb5omC7tcc1FgaXd3uCm1nJ9l+8mj14VBDYmG/CTvikO1ujggEhMIIBHTASBgNVHRMBAf8ECDAGAQH/AgEAMBcGA1UdIAEB/wQNMAswCQYHZ4ESAQIBAjBNBgNVHR8ERjBEMEKgQKA+hjxodHRwOi8vZ3NtYS1jcmwuc3ltYXV0aC5jb20vb2ZmbGluZWNhL2dzbWEtcnNwMi1yb290LWNpMS5jcmwwDgYDVR0PAQH/BAQDAgEGMDcGA1UdHgEB/wQtMCugKTAnpCUwIzEOMAwGA1UEChMFS2lnZW4xETAPBgNVBAUTCDg5MDQ0MDQ1MBYGA1UdEQQPMA2ICysGAQQBg7sTAQICMB0GA1UdDgQWBBQYmghTr9SU9rB5mgtn8PXQwSnj9jAfBgNVHSMEGDAWgBSBNw9RJdCx1AjUw7Iy5tJeeVvr+zAKBggqhkjOPQQDAgNHADBEAiAQZhiqj72l9CmSgfTjcSIE5unzsJXxm45S7aYffkpNsQIgNfgtDjRuLoNsfxhEOSo5LkPb3vpE4L8iFSXolOWTg9E="
data = base64.b64decode(authenticate_server_response)
response = db.types["AuthenticateServerResponse"]
db.encode("GetEuiccChallengeRequest", dict())

b'\xbf.\x00'

In [2]:
import base64
import binascii

b64_certificate = "MIICVDCCAfugAwIBAgIQZumBlNebG7wHl/5VrnPCxDAKBggqhkjOPQQDAjBEMRgwFgYDVQQKEw9HU00gQXNzb2NpYXRpb24xKDAmBgNVBAMTH0dTTSBBc3NvY2lhdGlvbiAtIFJTUDIgUm9vdCBDSTEwHhcNMjQwOTE4MDAwMDAwWhcNMjcwOTE3MjM1OTU5WjBCMSAwHgYDVQQKDBdSRURURUEgTU9CSUxFIFBURS4gTFRELjEeMBwGA1UEAwwVUkVEVEVBIE1PQklMRSBEUF9BVVRIMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEJ1JBe+EEsyocAQ9vYzjCBttMkGcufGsLiULDkIHcbGKiSBHRkZIPCI8S/Rhw6yB1BDW0IUA0FY4eRVdsqS2tNqOB0DCBzTAXBgNVHSABAf8EDTALMAkGB2eBEgECAQQwTQYDVR0fBEYwRDBCoECgPoY8aHR0cDovL2dzbWEtY3JsLnN5bWF1dGguY29tL29mZmxpbmVjYS9nc21hLXJzcDItcm9vdC1jaTEuY3JsMA4GA1UdDwEB/wQEAwIHgDATBgNVHREEDDAKiAgrBgEEAYPFHDAdBgNVHQ4EFgQUH3INoRziBpuXcwV1oNpnXGeVt6IwHwYDVR0jBBgwFoAUgTcPUSXQsdQI1MOyMubSXnlb6/swCgYIKoZIzj0EAwIDRwAwRAIgbq6dUp9M0LSTAVTgmiPixvn/lvcIoMnc7iJ13DLmZ4oCIHIdk6ZjtCVfahek5h8yQ0WdZ646hPRd5M3SpReTZNWJ"
certificate = base64.b64decode(b64_certificate)
decoded = rsp_v2_6.decode("Certificate", certificate)
encoded = rsp_v2_6.encode("Certificate", decoded)

hex_encoded = binascii.hexlify(encoded)
hex_certificate = binascii.hexlify(certificate)
assert hex_encoded == hex_certificate, (
    f"Decoded and encoded certificate do not match: {hex_certificate} != {hex_encoded}"
)
print(decoded)


AssertionError: Decoded and encoded certificate do not match: b'30820254308201fba003020102021066e98194d79b1bbc0797fe55ae73c2c4300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303931383030303030305a170d3237303931373233353935395a30423120301e060355040a0c17524544544541204d4f42494c45205054452e204c54442e311e301c06035504030c15524544544541204d4f42494c452044505f415554483059301306072a8648ce3d020106082a8648ce3d030107034200042752417be104b32a1c010f6f6338c206db4c90672e7c6b0b8942c39081dc6c62a24811d191920f088f12fd1870eb20750435b4214034158e1e45576ca92dad36a381d03081cd30170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302078030130603551d11040c300a88082b0601040183c51c301d0603551d0e041604141f720da11ce2